##### This notebook demonstrates a demo of how churn-prediction model can be trained and deployed on truefoundry platform.

# 🛠 Setup

In [ ]:
pip install -U "servicefoundry==0.6.6" "mlfoundry==0.6.1" 

In [ ]:
import logging
[logging.root.removeHandler(h) for h in logging.root.handlers]
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(name)s] %(levelname)-8s %(message)s')

In [ ]:
!sfy login --relogin

In [ ]:
# Copy workspace FQN from https://app.truefoundry.com/workspaces
from getpass import getpass

USERNAME="truefoundry-demo"
WORKSPACE="tfy-cluster-euwe1:demos"
TFY_API_KEY = getpass("Please enter your API Key,( https://app.truefoundry.com/settings ):-")

In [ ]:
!git clone https://github.com/truefoundry/truefoundry-examples.git
%cd truefoundry-examples
%cd end-to-end-examples/churn-prediction

# ⚡ Deploying a training Job - Quick Start
Here we will create a training job that will run on truefoundry's infrastructure and save the results.

### Training Script


In [ ]:
%cd train

In [53]:
!pygmentize main.py

import pandas as pd
from sklearn.neighbors import KNeighborsClassifier as Classification
import mlfoundry as mlf
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


def experiment_track(model, params, metrics, X_train, X_test):
    mlf_api = mlf.get_client()
    mlf_run = mlf_api.create_run(
        project_name="churn-prediction", run_name="churn-train-job"
    )
    mlf_run.log_params(params)
    mlf_run.log_metrics(metrics)
    mlf_run.log_dataset("train", X_train)
    mlf_run.log_dataset("test", X_test)
    model_version = mlf_run.log_model(
        name="churn-model",
        model=model,
        framework=mlf.ModelFramework.SKLEARN,
        description="churn-prediction-model",
    )
    mlf_run.log_plots({"confusion_matrix": plt}, step=1)
    return model_version.fqn


def train_model(hyperparams):
    df = pd.read_csv("https://raw.githubusercontent.com/nikp117

### Defining the requirements

In [54]:
!pygmentize requirements.txt

matplotlib==3.1.3
matplotlib-inline==0.1.6
mlfoundry>=0.6.1,<0.6.3
pandas==1.5.0


### Deploy as a training job

In [ ]:
import argparse
import logging
import os
from servicefoundry import Build, PythonBuild, Resources, Job, Param

logging.basicConfig(level=logging.INFO)

# Defining the job
job = Job(
    name="churn-prediction-train",
    image=Build(
        build_spec=PythonBuild(
            command="python main.py --n_neighbors {{n_neighbors}} --weights {{weights}} --algorithm {{algorithm}} --power {{power}}"
        )
    ),
    env={
        "TFY_API_KEY": TFY_API_KEY
    },
    params=[
        Param(name="n_neighbors", default='5', description="Number of neighbors to use by default"),
        Param(name="weights", default='uniform', description="Weight function used in prediction.  Possible values: uniform, distance"),
        Param(name="algorithm", default='auto', description="Algorithm used to compute the nearest neighbors: possible values: 'auto', 'ball_tree', 'kd_tree', 'brute'"),
        Param(name="power", default='2', description="Power parameter for the Minkowski metric. When p = 1, this is manhattan_dist, and euclidean_dist p = 2")
    ],
    resources=Resources(
        cpu_request=1, cpu_limit=1, memory_request=2000, memory_limit=2000,
    ),
)

# Finally, we call deploy to push it to Truefoundry platform
job.deploy(workspace_fqn=WORKSPACE)

### Trigger the training job from UI
Click on https://app.truefoundry.com/deployments?tab=jobs and trigger a run.

# ☁ Deploying the trained Model
Here we will deploy the model saved in truefoundry's model registry at the time of model training.

In [ ]:
import logging

from servicefoundry import ModelDeployment, Resources, TruefoundryModelRegistry

logging.basicConfig(level=logging.INFO)

MODEL_VERSION_FQN = f"model:truefoundry/{USERNAME}/churn-prediction/churn-model:1"

model_deployment = ModelDeployment(
    name=f"churn-prediction",
    model_source=TruefoundryModelRegistry(model_version_fqn=MODEL_VERSION_FQN),
    resources=Resources(cpu_request=0.2, cpu_limit=0.5, memory_request=500, memory_limit=1000)
)

model_deployment.deploy(workspace_fqn=WORKSPACE)

#  ☁  Deploying  Model's Demo as a Service
In this section we will deploy a training job that trains a SVM model on iris dataset

In [55]:
%cd ../demo

/Users/nikp18/Downloads/truefoundry-examples/end-to-end-examples/churn-prediction/demo/truefoundry-examples/end-to-end-examples/churn-prediction/train/truefoundry-examples/end-to-end-examples/churn-prediction/demo/truefoundry-examples/end-to-end-examples/churn-prediction/demo


### Creating a gradio demo

In [56]:
!pygmentize demo.py

import gradio as gr
import datetime
import os
import requests
from urllib.parse import urljoin

MODEL_DEPLOYED_URL = os.environ['MODEL_DEPLOYED_URL']

inputs = [
    gr.Number(label="CreditScore", value=619),
    gr.Number(label="Age", value=42),
    gr.Number(label="Tenure", value=2),
    gr.Number(label="Balance", value=0),
    gr.Number(label="NumOfProducts", value=1),
    gr.Number(label="HasCrCard", value=1),
    gr.Number(label="IsActiveMember", value=1),
    gr.Number(label="EstimatedSalary", value=101348.88)
]


def predict(*val):
    json_body = {"parameters": {
        "content_type": "pd"
    }, "inputs": []}

    for v, inp in zip(val, inputs):
        json_body["inputs"].append(
            {
                "name": inp.label,
                "datatype": "FP32",
                "data": [v],
                "shape": [1]
            }
        )
    resp = requests.post(
        url=urljoin(MODEL_DEPLOYED_URL, "v2/models/churn-model/infer"),
        json=json_body
    )
    r

### Defining the requirements

In [58]:
!pygmentize requirements.txt

requests==2.28.1
gradio==3.17.1
urllib3==1.26.12


### Deploying the grdaio demo as a 'Service'

In [ ]:
import logging

from servicefoundry import Build, PythonBuild, Resources, Service

logging.basicConfig(level=logging.INFO)

MODEL_DEPLOYED_URL = f"https://churn-prediction-{WORKSPACE.split(':')[-1]}.tfy-ctl-euwe1-production.production.truefoundry.com"

# creating a service object and defining all the configurations
service = Service(
    name="churn-prediction-demo",
    image=Build(
        build_spec=PythonBuild(
            command="python demo.py",
            python_version="3.9",
        ),
    ),
    env={
        "MODEL_DEPLOYED_URL": MODEL_DEPLOYED_URL,
    },
    ports=[{"port": 8080}], #In public cloud deployment TrueFoundry exposes port 8080    
    resources=Resources(
        cpu_request=0.5, cpu_limit=0.5, memory_limit=2500, memory_request=1500
    ),
    replicas=1
)
service.deploy(workspace_fqn=WORKSPACE)
